In [32]:
import numpy as np
import pandas as pd
import vectorbt as vbt
from numba import njit
from math import e

from vectorbt.generic.nb import diff_nb
from vectorbt.signals.factory import SignalFactory
from lib.utils import directory_to_data_frame_list, ExtendedPortfolio

In [33]:
directory = "/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/shorts/"
ohlcv_series_list = directory_to_data_frame_list(directory)

# concatenamos los dfs
names = list(map(lambda t: t[0], ohlcv_series_list))
dfs = list(map(lambda t: t[1].get(["Close", "Volume"]), ohlcv_series_list))
ov_df = pd.concat(dfs, axis=1, keys=names)
# borramos las filas que tengan nan(parece que algunos pueden estar desalineados)
ov_df.dropna(inplace=True)
ov_df.columns.set_names(["symbol", "value"], inplace=True)
ov_df.head()


symbol                   ADA                     BTC                  DASH  \
value                  Close        Volume     Close        Volume   Close   
date                                                                         
2021-03-16 22:09:00  1.22257  2.462095e+06  56699.99  2.686370e+06  223.98   
2021-03-16 22:10:00  1.22166  1.772561e+06  56732.66  3.413590e+06  224.03   
2021-03-16 22:11:00  1.22300  1.121237e+06  56700.00  2.683621e+06  223.97   
2021-03-16 22:12:00  1.22176  9.902460e+05  56748.48  1.538395e+06  224.26   
2021-03-16 22:13:00  1.21928  8.934922e+05  56650.12  1.672449e+06  224.15   

symbol                             
value                      Volume  
date                               
2021-03-16 22:09:00  19894.547668  
2021-03-16 22:10:00  18001.462307  
2021-03-16 22:11:00   2308.407769  
2021-03-16 22:12:00  17365.383226  
2021-03-16 22:13:00   1839.424917

In [34]:
# creamos el indicador para el lr y otro para el wlr
# lo hago por separado para poder calcular el mlr
# con data de varios activos del mercado
# y luego solo utiliza lr con los que me interesa
@njit
def lr_nb(close):
    c_log = np.log(close)
    lr = diff_nb(c_log)
    return lr

LR = vbt.IndicatorFactory(
    input_names=['close'],
    output_names=['lr']
).from_apply_func(lr_nb, use_ray=True)

@njit
def wlr_nb(volume, lr):
    mkt_vol = volume.sum(axis=1)
    mkt_ratio = (volume.T / mkt_vol).T
    wrl =  lr * mkt_ratio
    return wrl

WLR = vbt.IndicatorFactory(
    input_names=['volume', 'lr'],
    output_names=['wlr']
).from_apply_func(wlr_nb, use_ray=True)

In [35]:
close = ov_df.xs('Close', level='value', axis=1)
volume = ov_df.xs('Volume', level='value', axis=1)
lr_ind = LR.run(close)
wlr_ind = WLR.run(volume, lr_ind.lr)
mkt_lr = wlr_ind.wlr.sum(axis=1, skipna=False)
fig = mkt_lr.vbt.plot(trace_names=["MKT_LR"])

In [46]:
lr_ind.lr.head()

symbol,ADA,BTC,DASH
date,,,
2021-03-16 22:09:00,NaN,NaN,NaN
2021-03-16 22:10:00,-0.000745,0.000576,0.000223
2021-03-16 22:11:00,0.001096,-0.000576,-0.000268
2021-03-16 22:12:00,-0.001014,0.000855,0.001294
2021-03-16 22:13:00,-0.002032,-0.001735,-0.000491


In [36]:
lr_ind.lr.vbt.plot(fig=fig).show()

In [37]:
ada_ov_df = ov_df.xs('ADA', level='symbol', axis=1)
ada_ov_df["Close"].vbt.plot(trace_names=["ADA"]).show()

In [53]:
 #creamos el indicador para las bandas
@njit
def mkt_banda_nb(mkt_lr, upper_filter, lower_filter):
    upper = np.where(mkt_lr >= upper_filter, mkt_lr, np.nan)
    lower = np.where(mkt_lr <= -lower_filter, mkt_lr, np.nan)
    return upper, lower

MKT_BANDS = vbt.IndicatorFactory(
    input_names=['mkt_lr'],
    param_names=['upper_filter', 'lower_filter'],
    output_names=['upper', 'lower']
).from_apply_func(mkt_banda_nb, use_ray=True)

In [120]:
filters = np.linspace(0.00001, 0.005, 60, endpoint=False)
mkt_bands_ind = MKT_BANDS.run(mkt_lr=mkt_lr, upper_filter=filters , lower_filter=filters,
                        per_column=False,
                        param_product=True,
                        short_name="mkt")

In [121]:
ada_lr = lr_ind.lr["ADA"]
ada_close = ov_df.xs('Close', level='value', axis=1)["ADA"]
entries =  mkt_bands_ind.upper_above(ada_lr, crossover=True)
exits = mkt_bands_ind.lower_below(ada_lr, crossover=True)
#(entries*1 - 1*exits).vbt.plot().show()
portfolio_kwargs = dict(
    direction='longonly',
    freq='m',
)
port = ExtendedPortfolio.from_signals(ada_close, entries, exits, **portfolio_kwargs)

In [123]:
help(port.expected_log_returns().vbt.heatmap)

Help on method heatmap in module vectorbt.generic.accessors:

heatmap(x_level=None, y_level=None, symmetric=False, sort=True, x_labels=None, y_labels=None, slider_level=None, active=0, slider_labels=None, return_fig=True, fig=None, **kwargs) method of vectorbt.root_accessors.Vbt_SRAccessor instance
    Create a heatmap figure based on object's multi-index and values.
    
    If index is not a multi-index, converts Series into a DataFrame and calls `GenericDFAccessor.heatmap`.
    
    If multi-index contains more than two levels or you want them in specific order,
    pass `x_level` and `y_level`, each (`int` if index or `str` if name) corresponding
    to an axis of the heatmap. Optionally, pass `slider_level` to use a level as a slider.
    
    Creates `vectorbt.generic.plotting.Heatmap` and returns the figure.
    
    ## Example
    
    ```python-repl
    >>> multi_index = pd.MultiIndex.from_tuples([
    ...     (1, 1),
    ...     (2, 2),
    ...     (3, 3)
    ... ])
    >>> s

In [ ]:
data=port.expected_log_returns().vbt.heatmap().show()
port.sharpe_ratio().vbt.heatmap().show()

In [31]:
# un pequeño test
_py = pd.DataFrame({
    'Close': [1,e,e**2],
    'Volume': [1,2,1]
})
_thon = pd.DataFrame({
    'Close': [e**2,e,1],
    'Volume': [1,4,10]
})
_test_df = pd.concat([_py,_thon], axis=1, keys=["Py", "Thon"])
_test_df.columns.set_names(["asset", "value"], inplace=True)

close = _test_df.xs('Close', level='value', axis=1)
volume = _test_df.xs('Volume', level='value', axis=1)
_test_lrInd = LR.run(close)
_test_wlrInd = WLR.run(volume, _test_lrInd.lr)

exp_py_lr = np.array([np.nan, 1, 1])
exp_thon_lr = np.array([np.nan, -1, -1])
assert (np.allclose(exp_py_lr, _test_lrInd.lr["Py"], equal_nan=True))
assert (np.allclose(exp_thon_lr, _test_lrInd.lr["Thon"], equal_nan=True))
exp_py_vr = np.array([0.5, 1/3, 1/11])
exp_thon_vr = np.array([0.5, 2/3, 10/11])
exp_py_wlr = exp_py_lr * exp_py_vr
exp_thon_wlr = exp_thon_lr * exp_thon_vr
assert (np.allclose(exp_py_wlr, _test_wlrInd.wlr["Py"], equal_nan=True))
assert (np.allclose(exp_thon_wlr, _test_wlrInd.wlr["Thon"], equal_nan=True))
# falta testear el cálculo de mkt_lr
_test_mkt_lr = _test_wlrInd.wlr.sum(axis=1, skipna=False)
exp_mkt_lr = exp_py_wlr + exp_thon_wlr
assert (np.allclose(exp_mkt_lr,_test_mkt_lr, equal_nan=True))